<a href="https://colab.research.google.com/github/JuanDavidGit/analisis-exploratorio-de-datos/blob/main/Tarea_2_analisis_univariado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importamos las librerias que usaremos en este practica

import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#importamos la libreria de drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
# Modificamos la ruta de acuerdo a su Gdrive
#Usamos pd (la libreria de pandas, que fue definida en la celda donde importamos las librerias) para leer el dataset y asignarlo a df
df = pd.read_csv('/content/drive/MyDrive/analisis exploratorio/dataset_tringas.csv',    #Ruta completa a la ubicación del archivo de interés.
                 sep=";")              #Como el archivo contiene algunos caracteres ?, con este comando pandas los interpreta como nan (not a number).

In [6]:
df.shape

(120, 5)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   arrozal               120 non-null    int64
 1   arrozal con rotacion  120 non-null    int64
 2   arrozal organico      120 non-null    int64
 3   borde de rio          120 non-null    int64
 4   humedal               120 non-null    int64
dtypes: int64(5)
memory usage: 4.8 KB


In [ ]:
df.head()

,arrozal,arrozal con rotacion,arrozal organico,borde de rio,humedal
0,47,41,31,7,48
1,44,27,31,20,13
2,27,22,33,5,38
3,9,7,40,43,42
4,12,30,37,8,6


In [7]:
df.describe()

,arrozal,arrozal con rotacion,arrozal organico,borde de rio,humedal
count,120.000000,120.000000,120.000000,120.000000,120.00000
mean,30.766667,26.775000,26.716667,30.900000,28.67500
std,20.535983,21.385011,19.928942,20.465343,19.87235
min,0.000000,0.000000,0.000000,0.000000,0.00000
25%,15.750000,11.750000,11.000000,13.750000,14.75000
50%,27.500000,24.000000,26.000000,31.000000,26.50000
75%,41.250000,38.250000,38.250000,43.000000,38.00000
max,106.000000,97.000000,96.000000,100.000000,96.00000


In [ ]:
df['promedio_individuos'] = df[['arrozal', 'arrozal con rotacion', 'arrozal organico', 'borde de rio', 'humedal']].mean(axis=1)

In [ ]:
# Escojamos nuestras variables predictoras
predictores = ['arrozal', 'arrozal con rotacion', 'arrozal organico', 'borde de rio', 'humedal']

# Definir el objetivo como el promedio de individuos por hábitat
objetivo = 'promedio_individuos'

# Asignar las variables X (predictoras) e y (objetivo)
y = df[objetivo].values
X = df[predictores].values

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LinearRegression

# Crear el modelo de regresión lineal
linear_reg = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
linear_reg.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = linear_reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calcular el R²
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

Mean Squared Error: 2.5900933054756556e-29
R² Score: 1.0


In [ ]:
#ANOVA
from scipy.stats import f_oneway

# Supongamos que tienes 5 hábitats: arrozal, arrozal con rotacion, arrozal organico, borde de rio, humedal
anova_result = f_oneway(df['arrozal'], df['arrozal con rotacion'], df['arrozal organico'], df['borde de rio'], df['humedal'])
print(f'F-statistic: {anova_result.statistic}, p-value: {anova_result.pvalue}')


F-statistic: 1.2006575188847401, p-value: 0.30938079382921807


In [ ]:
!pip install statsmodels


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Convertir el DataFrame a formato largo (long format) para ANOVA y Tukey
df_melted = pd.melt(df, id_vars=[], value_vars=['arrozal', 'arrozal con rotacion', 'arrozal organico', 'borde de rio', 'humedal'])
df_melted.columns = ['habitat', 'individuos']

# Realizar el ANOVA
modelo = ols('individuos ~ C(habitat)', data=df_melted).fit()
anova_result = anova_lm(modelo, typ=2)
print(anova_result)

# Realizar el test de Tukey HSD
tukey_result = pairwise_tukeyhsd(endog=df_melted['individuos'], groups=df_melted['habitat'], alpha=0.05)
print(tukey_result)


                   sum_sq     df         F    PR(>F)
C(habitat)    2007.450000    4.0  1.200658  0.309381
Residual    248703.883333  595.0       NaN       NaN
               Multiple Comparison of Means - Tukey HSD, FWER=0.05               
       group1               group2        meandiff p-adj   lower    upper  reject
---------------------------------------------------------------------------------
             arrozal arrozal con rotacion  -3.9917 0.5548 -11.2135  3.2302  False
             arrozal     arrozal organico    -4.05 0.5404 -11.2719  3.1719  False
             arrozal         borde de rio   0.1333    1.0  -7.0885  7.3552  False
             arrozal              humedal  -2.0917 0.9328  -9.3135  5.1302  False
arrozal con rotacion     arrozal organico  -0.0583    1.0  -7.2802  7.1635  False
arrozal con rotacion         borde de rio    4.125 0.5219  -3.0969 11.3469  False
arrozal con rotacion              humedal      1.9 0.9519  -5.3219  9.1219  False
    arrozal organico 

Teniendo los resultados de los censos de Tinga flavipes en diferentes habitat se quizo obtener un promedio de individuos por habitat. Dado que es un resultado muy sencillo de obtener, el modelo da un R2 de 1, esto significa que el modelo explica el 100% de la variablidad de los datos, esto se debe a que hay una alta correlación entre las variables predictoras y el objetivo. Por otro lado un Mean Squared Error bajo indica que las predicciones del modelo son casi perfectas.

Hice otro analisis para obtener resultados no tan perfectos, usando un ANOVA se quizo compara los diferentes ecosistemas, dando como resultado un p-value>0.05, lo que significa que no hay diferencias significativas entre los ecosistemas, esto se corrobora con un post hoc, como resultado, el False nos indica que no hay diferecia significativa entre los diferentes ecosistemas, basicamente que todos se comportan igual en cuanto a numero de ind. (La base de datos fue generada con chat GPT ya que no se cuenta con la base de datos real)